In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import sampler
from torch.optim import SGD, Adam

import torchvision.models as models

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import PIL
from PIL import Image
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [4]:
batch_size    = 100
noise_size    = 296
size = 400
w = 80
dtype         = torch.FloatTensor

def sample_noise(batch_size, dim):
    return (torch.rand([batch_size, dim])*2 - 1).type(dtype)

class Flatten(nn.Module):
    
    def forward(self, x):
        N, C, H, W = x.size()
        return x.view(N, -1)  
    
class Unflatten(nn.Module):
    
    def __init__(self, N=-1, C=128, H=7, W=7):
        super(Unflatten, self).__init__()
        self.N = N
        self.C = C
        self.H = H
        self.W = W
    def forward(self, x):
        return x.view(self.N, self.C, self.H, self.W)

G = nn.Sequential(
            
            nn.Linear(noise_size,1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 8*w*w),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(8*w*w),
            Unflatten(batch_size, 128, w // 4, w // 4),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
            
        ).type(dtype)

In [6]:
device = torch.device('cpu')
G.load_state_dict(torch.load("AnimeGenerator_state_dict.pth", map_location=device))

<All keys matched successfully>

In [7]:
g_fake_seed = sample_noise(batch_size, noise_size)
fake_images = G(g_fake_seed)
plt.imshow(fake_images[12].permute(1, 2, 0).cpu().detach())

TypeError: sample_noise() missing 1 required positional argument: 'dim'